# 10k Books: Nearest Neighbors

*This notebook's original model resulted in title predictions, modifications
in this version use ISBN 13*
- using data found on kaggle from Goodreads
- books.csv contains information for 10,000 books, such as ISBN, authors, title, year
- ratings.csv is a collection of user ratings on these books, from 1 to 5 stars

In [2]:
import numpy as pd
import pandas as pd
import pickle


from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# TODO: convert model to use ISBN 10 (ISBN 13 is permanently unusable)

**Books dataset**

In [4]:
books = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv')

In [5]:
print(books.shape)
books.head()

(10000, 23)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [10]:
books.isnull().sum()

book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

In [6]:
books.iloc[2123]

book_id                                                                   2124
goodreads_book_id                                                        41193
best_book_id                                                             41193
work_id                                                                2234336
books_count                                                                 51
isbn                                                                1400033837
isbn13                                                              9.7814e+12
authors                                                            Jane Smiley
original_publication_year                                                 1991
original_title                                                A Thousand Acres
title                                                         A Thousand Acres
language_code                                                              eng
average_rating                                      

**Ratings dataset**

In [47]:
ratings = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv')

In [48]:
print(ratings.shape)
ratings.head()

(5976479, 3)


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


**Trim down the data**

_In order to make a user rating matrix we will only need bood_id and title._



In [49]:
cols = ['book_id', 'title']

books = books[cols]
books.head()

,book_id,title
0,1,"The Hunger Games (The Hunger Games, #1)"
1,2,Harry Potter and the Sorcerer's Stone (Harry P...
2,3,"Twilight (Twilight, #1)"
3,4,To Kill a Mockingbird
4,5,The Great Gatsby


**Clean up book titles**

_Book titles are messy, special characters, empty spaces, brackets clutter up the titles_

In [50]:
def clean_book_titles(title):
  title = re.sub(r'\([^)]*\)', '', title) # handles brackets
  title = re.sub(' + ', ' ', title) #compresses multi spaces into a single space
  title = title.strip() # handles special characters
  return title

In [51]:
books['title'] = books['title'].apply(clean_book_titles)

books.head()

,book_id,title
0,1,The Hunger Games
1,2,Harry Potter and the Sorcerer's Stone
2,3,Twilight
3,4,To Kill a Mockingbird
4,5,The Great Gatsby


**neat-o**

**Create feature matrix**

_Combine datasets to get a new dataset of user ratings for each book_

In [52]:
books_ratings = pd.merge(ratings, books, on='book_id')

print(books_ratings.shape)

books_ratings.head()


(5976479, 4)


,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind
1,11,258,3,The Shadow of the Wind
2,143,258,4,The Shadow of the Wind
3,242,258,5,The Shadow of the Wind
4,325,258,4,The Shadow of the Wind


**Remove rows with same user_id and book title**



In [53]:
user_ratings = books_ratings.drop_duplicates(['user_id', 'title'])

print(user_ratings.shape)
user_ratings.head()

(5972713, 4)


,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind
1,11,258,3,The Shadow of the Wind
2,143,258,4,The Shadow of the Wind
3,242,258,5,The Shadow of the Wind
4,325,258,4,The Shadow of the Wind


**Pivot table to create user_ratings matrix**

_Each column is a user and each row is a book. The entries in the martix are the user's rating for that book._

In [54]:
user_matrix = user_ratings.pivot(index='title', columns='user_id', values='rating').fillna(0)

user_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
title,,,,,,,,,,,,,,,,,,,,,
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#GIRLBOSS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
'Tis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1,000 Places to See Before You Die",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
user_matrix.shape

(9800, 53424)

**Compress the matrix since it is extremely sparse**

_Whole lotta zeros_

_

In [56]:
compressed = csr_matrix(user_matrix.values)

In [57]:
# build and train knn

# unsupervised learning
# using cosine to measure space/distance

knn = NearestNeighbors(algorithm='brute', metric='cosine')

knn.fit(compressed)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [58]:
def get_recommendations(book_title, matrix=user_matrix, model=knn, topn=2):
    book_index = list(matrix.index).index(book_title)
    distances, indices = model.kneighbors(matrix.iloc[book_index,:].values.reshape(1,-1), n_neighbors=topn+1)
    print('Recommendations for {}:'.format(matrix.index[book_index]))
    for i in range(1, len(distances.flatten())):
        print('{}. {}, distance = {}'.format(i, matrix.index[indices.flatten()[i]], "%.3f"%distances.flatten()[i]))
    print()
    
get_recommendations("Harry Potter and the Sorcerer's Stone")
get_recommendations("Pride and Prejudice")
get_recommendations("Matilda")

Recommendations for Harry Potter and the Sorcerer's Stone:
1. Harry Potter and the Prisoner of Azkaban, distance = 0.320
2. Harry Potter and the Chamber of Secrets, distance = 0.327

Recommendations for Pride and Prejudice:
1. Jane Eyre, distance = 0.421
2. Sense and Sensibility, distance = 0.434

Recommendations for Matilda:
1. The Witches, distance = 0.501
2. The BFG, distance = 0.525



In [59]:
pickle.dump(knn, open('knn_model.pkl','wb'))
